In [ ]:
%load_ext autoreload
%autoreload 2

import ipywidgets as w
w.Widget.close_all()

from rotating_coil_analyzer.gui.app import build_catalog_gui
gui = build_catalog_gui()
gui

In [ ]:
import os
import re

from rotating_coil_analyzer.ingest.readers_sm18 import Sm18CorrSigsReader

# 1) Point to the exact file you are debugging (example: Seg3)
fpath = r"C:\Local data\2025_12_SM18test\HCMCBRD_S001-CR000012_20240315_093813_AP1-0_AP2-Stair_Step\aperture1\HCMCBRD_S001-CR000012_20240315_093813_AP1-0_AP2-Stair_Step_corr_sigs_Ap_1_Seg3.bin"

# 2) Run id (usually the prefix of your file/folder; keep consistent)
run_id = "HCMCBRD_S001-CR000012_20240315_093813_AP1-0_AP2-Stair_Step"

# 3) Segment from filename (fallback to "3" if parsing fails)
m = re.search(r"_Seg(\d+)\.bin$", os.path.basename(fpath))
segment = m.group(1) if m else "3"

# 4) Acquisition settings (from Parameters.txt)
Ns = 512
shaft_speed_rpm = 60.0  # use abs(v); sign doesn't matter for period

reader = Sm18CorrSigsReader()
seg_frame = reader.read(
    fpath,
    run_id=run_id,
    segment=segment,
    samples_per_turn=Ns,
    shaft_speed_rpm=shaft_speed_rpm,
)

seg_frame, seg_frame.df.head()


In [ ]:
import numpy as np

df = seg_frame.df
Ns = seg_frame.samples_per_turn
n_turns = len(df) // Ns
sl = slice(0, n_turns*Ns)

abs_mat = df["df_abs"].to_numpy()[sl].reshape(n_turns, Ns)
abs_rms = np.sqrt(np.mean(abs_mat**2, axis=1))  # per-turn amplitude proxy

cand = [c for c in ["I", "I0", "I1", "I2"] if c in df.columns]
print("candidates:", cand)

for c in cand:
    x = df[c].to_numpy()[sl].reshape(n_turns, Ns)
    per_turn_med = np.median(x, axis=1)
    within_turn_std_med = float(np.median(np.std(x, axis=1)))

    corr = np.corrcoef(per_turn_med, abs_rms)[0, 1]
    p05, p995 = np.percentile(per_turn_med, [0.5, 99.5])

    print(f"{c:>3s}  within-turn std~{within_turn_std_med:.4g}   "
          f"turn-med range(p99.5-p0.5)~{(p995-p05):.4g}   corr(abs_rms)={corr:.3f}")


In [ ]:
import numpy as np

sl = slice(0, (len(seg_frame.df)//seg_frame.samples_per_turn)*seg_frame.samples_per_turn)
for c in ["I", "I1", "I2"]:
    if c in seg_frame.df.columns:
        print(c, float(np.nanmax(np.abs(seg_frame.df[c].to_numpy()[sl] - seg_frame.df["I"].to_numpy()[sl]))))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df = seg_frame.df
Ns = seg_frame.samples_per_turn
n_turns = len(df)//Ns

t_turn = df["t"].to_numpy()[0:n_turns*Ns:Ns]  # one timestamp per turn
for c in ["I0","I1","I2","I"]:
    if c in df.columns:
        x = df[c].to_numpy()[:n_turns*Ns].reshape(n_turns, Ns)
        plt.figure()
        plt.plot(t_turn, np.median(x, axis=1))
        plt.title(f"per-turn median of {c}")
        plt.xlabel("t_turn [s]")
        plt.ylabel(c)
        plt.show()


In [ ]:
import numpy as np

df = seg_frame.df

# Sample-level correlation (often strong magnitude)
corr = np.corrcoef(df["df_abs"].to_numpy(), df["df_cmp"].to_numpy())[0,1]
print("corr(df_abs, df_cmp) =", corr)

# RMS ratio (cmp should be much smaller than abs)
rms_abs = np.sqrt(np.mean(df["df_abs"].to_numpy()**2))
rms_cmp = np.sqrt(np.mean(df["df_cmp"].to_numpy()**2))
print("rms_cmp/rms_abs =", rms_cmp/rms_abs)


In [ ]:
I1 = df["I1"].to_numpy()
I2 = df["I2"].to_numpy()

print("corr(I1, I2) =", np.corrcoef(I1, I2)[0,1])
print("max|I1-I2| =", np.max(np.abs(I1 - I2)))
print("std(I1-I2) =", np.std(I1 - I2))


# Test

In [2]:
from pathlib import Path
import numpy as np

from rotating_coil_analyzer.ingest.discovery import MeasurementDiscovery
from rotating_coil_analyzer.ingest.readers_sm18 import Sm18CorrSigsReader, Sm18ReaderConfig
from rotating_coil_analyzer.ingest.readers_mba import MbaRawMeasurementReader, MbaReaderConfig

ROOTS = [
    Path(r"C:/Local data/2025_12_19-analyzer_tests/867_Alberto/20251212_171026_SPS_MBA/20251212_171620_MBA"),
    # Path(r"PASTE_PATH_TO_A_REAL_DATASET_FOLDER_2"),
]

def check_segment(cat, run_id, ap_ui, seg_id):
    f = cat.get_segment_file(run_id, ap_ui, seg_id)
    ap_phys = cat.resolve_aperture(ap_ui)

    if f.name.lower().endswith("_raw_measurement_data.txt"):
        segf = MbaRawMeasurementReader(MbaReaderConfig()).read(
            f, run_id=run_id, segment=str(seg_id),
            samples_per_turn=cat.samples_per_turn, aperture_id=ap_phys
        )

        # MBA: No synthetic time. Raw 't' may reset across plateaus; check plateau-safe turns.
        assert "plateau_id" in segf.df.columns, "MBA must provide plateau_id"
        Ns = segf.samples_per_turn
        pid = segf.df["plateau_id"].to_numpy()
        jumps = np.where(np.diff(pid) != 0)[0]
        for j in jumps:
            assert ((j + 1) % Ns) == 0, f"Plateau boundary not aligned to turn boundary at sample {j+1}"

        # Optional diagnostic: within each plateau, time should be non-decreasing (warn-level check here)
        t = segf.df["t"].to_numpy()
        for p in np.unique(pid):
            mask = (pid == p)
            tt = t[mask]
            dt = np.diff(tt)
            if not (np.all(np.isfinite(dt)) and np.all(dt >= 0)):
                print(f"WARNING: non-monotonic t within plateau_id={int(p)} in {f.name}")

    else:
        segf = Sm18CorrSigsReader(Sm18ReaderConfig(strict_time=True)).read(
            f, run_id=run_id, segment=str(seg_id),
            samples_per_turn=cat.samples_per_turn, shaft_speed_rpm=cat.shaft_speed_rpm,
            aperture_id=ap_phys
        )

        # SM18: 't' is raw file time; strict_time=True requires strictly increasing.
        t = segf.df["t"].to_numpy()
        dt = np.diff(t)
        assert np.all(np.isfinite(dt)) and np.all(dt > 0), "SM18 time must be strictly increasing"

    # Common invariants
    assert len(segf.df) == segf.n_turns * segf.samples_per_turn
    for col in ["t", "df_abs", "df_cmp", "I"]:
        assert col in segf.df.columns, f"Missing required column {col}"

    return f, segf

for root in ROOTS:
    print("\n=== ROOT ===", root)
    cat = MeasurementDiscovery(strict=True).build_catalog(root)
    print("runs:", cat.runs)
    print("aps:", cat.logical_apertures)
    print("segments:", [(s.aperture_id, s.segment_id) for s in cat.segments])

    # Check up to a few segments (so it’s not too slow)
    n_checked = 0
    for run_id in cat.runs[:3]:
        for ap_ui in cat.logical_apertures[:2]:
            segs = cat.segments_for_aperture(ap_ui)
            for s in segs[:3]:
                f, segf = check_segment(cat, run_id, ap_ui, s.segment_id)
                print("OK:", f.name, "| n_turns:", segf.n_turns, "| Ns:", segf.samples_per_turn, "| n_samples:", len(segf.df))
                if segf.warnings:
                    print("  warnings:")
                    for w in segf.warnings[:12]:
                        print("   -", w)
                    if len(segf.warnings) > 12:
                        print(f"   - ... ({len(segf.warnings)-12} more)")
                n_checked += 1
                if n_checked >= 10:
                    break
            if n_checked >= 10:
                break
        if n_checked >= 10:
            break

print("\nLayer C smoke run completed.")



=== ROOT === C:\Local data\2025_12_19-analyzer_tests\867_Alberto\20251212_171026_SPS_MBA\20251212_171620_MBA
runs: ('20251212_171620_MBA',)
aps: [None]
segments: [(1, 'NCS'), (1, 'CS')]
OK: 20251212_171620_MBA_Run_00_I_0.00A_NCS_raw_measurement_data.txt | n_turns: 3502 | Ns: 1024 | n_samples: 3586048
  warnings:
   - MBA reader: concatenating 125 plateau files for base='20251212_171620_MBA', segment='NCS'
   - large concatenated MBA trace: 3586048 rows (preview/plotting may be slow).
   - current candidate ranges (p99.5-p0.5): col3:3105.8, col4:0.48658
   - selected main current column: col3 (stored as df['I'])
OK: 20251212_171620_MBA_Run_00_I_0.00A_CS_raw_measurement_data.txt | n_turns: 3502 | Ns: 1024 | n_samples: 3586048
  warnings:
   - MBA reader: concatenating 125 plateau files for base='20251212_171620_MBA', segment='CS'
   - large concatenated MBA trace: 3586048 rows (preview/plotting may be slow).
   - current candidate ranges (p99.5-p0.5): col3:3105.8, col4:0.48658
   - sele

In [5]:
import numpy as np

pid = 66
sub = segf.df[segf.df["plateau_id"] == pid]
t = sub["t"].to_numpy()

dt = np.diff(t)
print("plateau_id:", pid)
print("plateau_step:", int(sub["plateau_step"].iloc[0]))
print("plateau_I_hint:", float(sub["plateau_I_hint"].iloc[0]))
print("n_samples:", len(sub))
print("t finite:", np.sum(np.isfinite(t)), "/", len(t))
print("dt finite:", np.sum(np.isfinite(dt)), "/", len(dt))
print("min dt (finite):", np.min(dt[np.isfinite(dt)]))
print("n dt < 0:", np.sum((dt[np.isfinite(dt)]) < 0))
print("n non-finite dt:", np.sum(~np.isfinite(dt)))

# Where does it happen?
bad = np.where((~np.isfinite(dt)) | (dt < 0))[0]
print("first bad indices:", bad[:20])


plateau_id: 66
plateau_step: 66
plateau_I_hint: 2900.0
n_samples: 28672
t finite: 27648 / 28672
dt finite: 27647 / 28671
min dt (finite): 0.0009360000003653113
n dt < 0: 0
n non-finite dt: 1024
first bad indices: [27647 27648 27649 27650 27651 27652 27653 27654 27655 27656 27657 27658
 27659 27660 27661 27662 27663 27664 27665 27666]
